In [16]:
# Use seaborn for pairplot



!pip install -q git+https://github.com/tensorflow/docs
# Use some functions from tensorflow_docs


> Blockquote Initializing the dl

In [1]:

from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
import tensorboard
import pathlib
from  IPython import display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import numpy as np

import shutil
import tempfile


# Getting the file

In [14]:
logdir = pathlib.Path(tempfile.mkdtemp())/"tensorboard_logs"
shutil.rmtree(logdir, ignore_errors=True)

data_file = 'C:/Users/cmedders/Data/MOPS.csv'
data_raw = pd.read_csv(data_file)
df = data_raw.copy()






#FEATURES = 3

#ds = tf.data.experimental.CsvDataset(data,[float(),]*(FEATURES+1))



#def pack_row(*row):
#    label = row[0]
#    features = tf.stack(row[1:],1)
#    return features, label

In [15]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10)
        

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

# Create an instance of the model
    model = MyModel()

NameError: name 'MyModel' is not defined

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

In [ ]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [ ]:
@tf.function    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
def train_step(images, labelss):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
     
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

In [ ]:
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)
    

In [ ]:
EPOCHS = 5

for epoch in range(EPOCHS):
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()


    for images, labels in trainds:
        train_step(images, labels)
    for test_images, test_labels in testds:
        test_step(test_images, test_labels)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch+1,
                          train_loss.result(),
                          train_accuracy.result()*100,
                          test_loss.result(),
                          test_accuracy.result()*100))

In [ ]:
def get_compiled_model():
    model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1)
  ])

    model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
    return model

In [ ]:
model = get_compiled_model()
model.fit(trainds, epochs=15)

In [ ]:
inputs = {key: tf.keras.layers.Input(shape=(), name=key) for key in df.keys()}
x = tf.stack(list(inputs.values()), axis=-1)
x = tf.keras.layers.Dense(10, activation='relu')(x)
output = tf.keras.layers.Dense(1)(x)

model_func = tf.keras.Model(inputs=inputs, outputs=output)

model_func.compile(optimizer='adam',
                   loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=['accuracy'])

In [ ]:
dict_slices = tf.data.Dataset.from_tensor_slices((df.to_dict('list'), pka.values)).batch(16)

In [ ]:
for dict_slice in dict_slices.take(1):
    print (dict_slice)

In [ ]:
model_func.fit(dict_slices, epochs=15)

# Splitting the file up so that testing has verifiable data

In [5]:
inputs_test = df.sample(frac=1/6,random_state=0)
outputs_test = df.drop(inputs_test.index)
outputs_test.tail()
#packed_ds = ds.batch(60).map(pack_row).unbatch()

N_VALIDATION = 10
N_TRAIN = 60
BUFFER_SIZE = 60
BATCH_SIZE = 5
STEPS_PER_EPOCH = N_TRAIN//BATCH_SIZE

# Checking out the stats of dataset

In [6]:

input_stats = inputs_test.describe()
input_stats.pop("pKa")
input_stats = input_stats.transpose()
input_stats

input_labels = inputs_test.pop("pKa")
outputs_labels =  outputs_test.pop("pKa")
#validate_ds = packed_ds.take(N_VALIDATION).cache()
#train_ds = packed_ds.skip(N_VALIDATION).take(N_TRAIN).cache()
#validate_ds = validate_ds.batch(BATCH_SIZE)
#train_ds = train_ds.shuffle(BUFFER_SIZE).repeat().batch(BATCH_SIZE)

# Manipulating Data

In [7]:

lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
  0.01,
  decay_steps=STEPS_PER_EPOCH*10,
  decay_rate=1,
  staircase=False)

def get_optimizer():
    return tf.keras.optimizers.Adam(lr_schedule)

In [8]:
def norm(x):
    return(x - input_stats['mean'] / input_stats['std'])
normed_input_data = norm(inputs_test)
normed_test_data = norm(outputs_test)
normed_test_data.tail()
def get_callbacks(name):
    return [
        tfdocs.modeling.EpochDots(),
        tf.keras.callbacks.TensorBoard(logdir/name),
    ]

# Building Model

In [9]:
#def build_model():
#    model = keras.Sequential([
#        layers.Dense(64, activation='relu', input_shape=[len(inputs_test.keys())]),
#        layers.Dropout(0.5),
#        layers.Dense(64, activation='relu', kernel_),
#        layers.Dropout(0.5),
#        layers.Dense(64, activation='relu'),
#        layers.Dropout(0.5),
#        layers.Dense(64, activation='relu'),
#        layers.Dropout(0.5),
#        layers.Dense(1)
#    ])
#    
#    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.95, beta_2=0.999, amsgrad=False)
    
#    model.compile(loss='mse',
#                 optimizer=optimizer,
#                 metrics=['mae', 'mse'])
#    return model
def compile_and_fit(model, name, optimizer=None, max_epochs=10000):
    if optimizer is None:
        optimizer = get_optimizer()
        model.compile(optimizer=optimizer,
                          loss='mse',
                          metrics=['mae', 'mse'])
        model.summary()
        
        history = model.fit(
            normed_input_data,input_labels,
            epochs=max_epochs,
            validation_split= 0.2,
            callbacks=get_callbacks(name),
            verbose=0)
        return history

In [11]:
#model = build_model()
regularizer_histories = {}

dropout_model = tf.keras.Sequential([
    layers.Dense(8, kernel_regularizer=regularizers.l2(0.0001),
                 activation='relu', input_shape=(3,)),
    layers.Dropout(0.5),
    layers.Dense(8, kernel_regularizer=regularizers.l2(0.0001),
                 activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(8, kernel_regularizer=regularizers.l2(0.0001),
                 activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(8, kernel_regularizer=regularizers.l2(0.0001),
                 activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1)
])

regularizer_histories['dropout'] = compile_and_fit(dropout_model, "regularizers/dropout")

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 32        
_________________________________________________________________
dropout (Dropout)            (None, 8)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 72        
_________________________________________________________________
dropout_1 (Dropout)          (None, 8)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 72        
_________________________________________________________________
dropout_2 (Dropout)          (None, 8)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 7

ValueError: Error when checking input: expected dense_input to have shape (3,) but got array with shape (5,)

In [ ]:
#model.summary()


In [ ]:
#example_batch = normed_input_data[0:10]
#example_result = model.predict(example_batch)
#example_batch.tail()
%tensorboard --logdir {logdir}/regularizers

display.IFrame(
    src="https://tensorboard.dev/experiment/fGInKDo8TXes1z7HQku9mw/#scalars&_smoothingWeight=0.97",
    width = "100%",
    height="800px")
tensorboard dev upload --logdir  {logdir}/regularizers

# Testing Model

In [ ]:
EPOCHS=10000
history = model.fit(
  normed_input_data, input_labels,
  epochs=EPOCHS, validation_split = 0.4, verbose=0,
  callbacks=[tfdocs.modeling.EpochDots()])

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()
plotter = tfdocs.plots.HistoryPlotter(smoothing_std=2)
plotter.plot({'Basic': history}, metric = "mae")
plt.ylim([0, .2])
plt.ylabel('MAE [pKa]')

In [ ]:
plotter.plot({'Basic': history}, metric = "mse")
plt.ylim([0, 0.05])
plt.ylabel('MSE [pKa^2]')

In [ ]:
model = build_model()

# The patience parameter is the amount of epochs to check for improvement
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

early_history = model.fit(normed_input_data, input_labels, 
                    epochs=EPOCHS, validation_split = .5, verbose=0, 
                    callbacks=[early_stop, tfdocs.modeling.EpochDots()])

In [ ]:
plotter.plot({'Early_History': early_history}, metric = "mae")
plt.ylim([0,3])
plt.ylabel('MAE[pKa]')

In [ ]:
loss, mae, mse = model.evaluate(normed_test_data, output_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} pKa".format(mae))

# Uneccessary corrections ... ?

In [ ]:
test_predictions = model.predict(normed_test_data).flatten()

a = plt.axes(aspect='equal')
plt.scatter(output_labels, test_predictions)
plt.xlabel('True Values [pKa]')
plt.ylabel('Predictions [pKa]')
lims = [4.5, 5]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims, lims)




In [ ]:
from tensorflow.keras.models import model_from_json
error = test_predictions - output_labels
plt.hist(error, bins = 15)
plt.xlabel("Prediction Error [pKa]")
_ = plt.ylabel("Count")
acetatesmodel_json = model.to_json()
with open("mesmodel.json", "w") as json_file:
    json_file.write(acetatesmodel_json)
# serialize weights to HDF5
model.save_weights("mesmodel.h5")

# Fixed it Yay so best of both worlds time, you can play with the model if you delete the stuff about saving a model and loading it (i knoooow so hard) at least they are just at the end of ^ cell and the beginning of the next one. Inputs are the little boxes at the end output is def ugly but functional

In [13]:
json_file = open('mesmodel.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
from ipywidgets import interact, interactive, fixed, interactive_output
from ipywidgets import FloatText, Output, HBox
x = FloatText(value=0.100,
    min=0.001,
    max=3.000,
    step=0.001,
    description='Total buffer molarity:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    )
y = FloatText(value=4.76,
    min=3.00,
    max=6.00,
    step=0.01,
    description='pH:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    )
z = FloatText(value=0,
    min=0.000,
    max=5.000,
    step=0.001,
    description='NaCl molarity:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    )
ui = HBox([x,y,z])
def f(x,y,z):    
    t = [[z,y,x]]
    df= pd.DataFrame(t,['NACL', 'pH', 'TOT'])
    normdf = norm(df)
    answer = loaded_model.predict(normdf)
    dif= df.iloc[0,1]-answer[0]
    molratio = np.power(10,dif)
    mwA = 82.03
    mHA = 17.4
    mwNaCl= 58.44
    mol_of_HA = df.iloc[0,2]/(molratio + 1)
    mL_of_HA = mol_of_HA/17.4*1000
    mol_A = df.iloc[0,2]-mol_of_HA
    g_of_A = mol_A*mwA
    g_of_NaCl = mwNaCl * z
    return(print(g_of_A,"g of Sodium Acetate Anhydrous and ", mL_of_HA, "mLs of Glacial Acetic Acid and ", g_of_NaCl, "g of NaCl", sep=""))
out = interactive_output(f, {'x':x, 'y':y, 'z':z})
display(ui, out) 

    
    

NameError: name 'model_from_json' is not defined